In [1]:
%config Completer.use_jedi = False
import sys
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import time
import threading

GPU = False
import os

if GPU:
    txt_device = 'gpu:0'
else:
    txt_device = 'cpu:0'    
    os.environ["CUDA_VISIBLE_DEVICES"]="-1"


In [68]:
class NN(tf.keras.layers.Layer):
    
    def __init__(self, layers, **kwargs):
        """
        
           layers: input, dense layers and outputs dimensions  
        """
        super().__init__(**kwargs)
        self.layers = layers
        self.num_layers = len(self.layers)        
        
    def build(self, input_shape):         
        """Create the state of the layers (weights)"""
        weights = []
        biases = []        
        for l in range(0,self.num_layers-1):
            W = self.xavier_init(size=[self.layers[l], self.layers[l+1]])
            b = tf.Variable(tf.zeros([1,self.layers[l+1]], dtype=tf.float64), dtype=tf.float64)
            weights.append(W)
            biases.append(b)
        
        self.Ws = weights
        self.bs = biases
        
    def xavier_init(self, size):
        in_dim = size[0]
        out_dim = size[1]        
        xavier_stddev = np.sqrt(2/(in_dim + out_dim))
        return tf.Variable(tf.compat.v1.truncated_normal([in_dim, out_dim], 
                                                         stddev=xavier_stddev, 
                                                         dtype=tf.float64), 
                           dtype=tf.float64)
    
    def normalise_input(self, inputs):
        """Map the inputs to the range [-1, 1]"""
        return 2.0*(inputs - self.lb)/(self.ub - self.lb) - 1.0
    
    @tf.function
    def __net__(self, inputs):
        H = inputs
        for W, b in zip(self.Ws[:-1], self.bs[:-1]):
            H = tf.tanh(tf.add(tf.matmul(H, W), b))
            
            W = self.Ws[-1]
            b = self.bs[-1]
            outputs = tf.add(tf.matmul(H, W), b)
        return outputs
    
    def call(self, inputs, grads=True):
        """Defines the computation from inputs to outputs"""
        X = tf.cast(inputs, tf.float64)
        outputs = self.__net__(X)
        if grads:                        
            partials_1 = [tf.gradients(outputs[:,i], X)[0] for i in range(outputs.shape[1])]
            partials_2 = [tf.gradients(partials_1[i], X)[0] for i in range(outputs.shape[1])]
            return outputs, partials_1, partials_2            
        else:            
            return outputs   
    
    


class TINN():
    """Turing-Informed Neoral Net"""
    def __init__(self, 
                 layers, 
                 inputs_obs,
                 output_obs,
                 inputs_pde = None,
                 boundary_spec = dict(
                    inputs_boundary = None,
                    boundary_loss_callback = None,
                    needs_grad = False 
                   )                 
                 ):
        self.model = NN(layers)
        self.inputs_obs = inputs_obs
        self.output_obs = output_obs        
        # Use the observation points for validating
        # PDE
        if inputs_pde is None:
            self.inputs_pde = inputs_obs
        else:
            self.inputs_pde = inputs_pde
            
        self.boundary_spec = boundary_spec
        if boundary_spec['inputs_boundary'] is None:
            self.has_bounday = False
        else:
            self.has_bounday = True
        if self.boundary_spec['boundary_loss_callback'] is None:
            self.boundary_loss_callback = self.periodic_boundary
        else:
            self.boundary_loss_callback = self.boundary_spec['boundary_loss_callback']        
        
        self.optimizer_Adam = keras.optimizers.Adam()
        #self.optimizer_Adam = tf.train.AdamOptimizer()
        #self.train_op_Adam = self.optimizer_Adam.minimize(self.loss)
     
    def periodic_boundary(self, inputs, boundary_pred, boundary_spec):        
        pass
    
    def pde_residuals(self, pde_inputs, pde_outputs, partials_1, partials_2):
        pass
    
    def model_vars(self):
        return []
    
    def loss_obs(self, inputs, outputs):
        obs_pred = self.model(inputs, grads = False)
        L = tf.reduce_sum(tf.square(obs_pred - outputs), name = "Loss_observations")
        return L
    
    def loss_pde(self, inputs):
        pde_outputs, partials_1, partials_2 = self.model(inputs, grads = True)
        pde_res = self.pde_residuals(inputs, pde_outputs, partials_1, partials_2)
        L = tf.reduce_sum(tf.square(pde_res), name = "Loss_pde")
        return L
        
    def loss_boundary(self, inputs, outputs):
        boundary_pred = self.model(inputs, self.boundary_spec['needs_grad'])        
        L = self.boundary_loss_callback(inputs, boundary_pred, self.boundary_spec)        
        return L
        
    
        
    def __batches__(self, batch_size):
        """Generator that returned shuffled indeces for each batch"""
        
        flg_boundary = self.has_bounday
        # Observation batch info 
        obs_n = self.inputs_obs.shape[0]
        batch_steps = obs_n//batch_size
        batch_steps = batch_steps + (obs_n-1)//(batch_steps*batch_size)
        # PDE batch info
        pde_n = self.inputs_pde.shape[0]
        pde_batch_size = pde_n//batch_steps        
        # Boundary condition batch info
        if flg_boundary:
            boundary_n = self.boundary_spec.inputs_boundary.shape[0]        
            boundary_batch_size = boundary_n//batch_steps
        # Observation indices   
        indices_obs = np.array(list(range(obs_n)))
        np.random.shuffle(indices_obs)
        # PDE  indices
        indices_pde = np.array(list(range(pde_n)))        
        np.random.shuffle(indices_pde)
        # Boundary condition  indices
        if flg_boundary:
            indices_boundary = np.array(list(range(boundary_n)))
            np.random.shuffle(indices_boundary)
            
        for batch in range(batch_steps):
            # Observation start-end
            obs_start = batch*batch_size
            obs_end = (batch+1)*batch_size
            obs_end = obs_end - (obs_end//obs_n)*(obs_end%obs_n)
            # PDE  start-end
            pde_start = batch*pde_batch_size
            pde_end = (batch+1)*pde_batch_size            
            # Boundary condition  start-end
            if flg_boundary:
                boundary_start = batch*boundary_batch_size
                boundary_end = (batch+1)*boundary_batch_size
                
            # Correction for PDE and boundary batches at last step
            if batch == batch_steps-1:
                if pde_end < pde_n:
                    pde_end = pde_n
                        
                if flg_boundary and boundary_end < boundary_n:
                    boundary_end = boundary_n
            # step's indices        
            batch_indices_obs = indices_obs[obs_start:obs_end]
            batch_indices_pde = indices_pde[pde_start:pde_end]
            if flg_boundary:
                batch_indices_boundary = indices_boundary[boundary_start:boundary_end]
                yield (batch_indices_obs, batch_indices_pde, batch_indices_boundary)
            else:
                yield (batch_indices_obs, batch_indices_pde, None)
        
    def train(self, epochs, batch_size, print_iter=10):
        start_time = time.time()
        for epoch in range(epochs):
            total_loss = 0            
            for obs_indeces, pde_indeces, boundary_indices in self.__batches__(batch_size):                
                obs_inputs_batch  = self.inputs_obs[obs_indeces]
                obs_outputs_batch = self.output_obs[obs_indeces]
                pde_inputs_batch  = self.inputs_pde[pde_indeces] 
                if self.has_bounday:
                    boundary_inputs_batch = self.boundary_spec.inputs_boundary[boundary_indices]
                    L = self.train_step(obs_inputs_batch, 
                                        obs_outputs_batch, 
                                        pde_inputs_batch,
                                        boundary_inputs_batch)
                else:
                    L = self.train_step(obs_inputs_batch, 
                                        obs_outputs_batch, 
                                        pde_inputs_batch,
                                        None)
                total_loss += L
                
            if epoch % print_iter == 0:
                elapsed = time.time() - start_time                                                                
                print(f"Epoch: {epoch}, loss:{total_loss:.2f}, \n"
                      f"Time:{elapsed:.2f}\n")
                start_time = time.time()
    
    @tf.function
    def train_step(self, 
                   obs_inputs_batch, 
                   obs_outputs_batch, 
                   pde_inputs_batch,
                   boundary_inputs_batch):
        
        with tf.GradientTape() as tape:
            loss_obs = self.loss_obs(obs_inputs_batch, obs_outputs_batch)
            loss_pde = self.loss_pde(pde_inputs_batch)
                    
            loss = loss_obs + loss_pde
            if self.has_bounday:
                loss += self.loss_boundary(boundary_inputs_batch)
                
        
        trainable_vars = self.model.trainable_weights + self.model_vars()
        grads = tape.gradient(loss,  trainable_vars)
        self.optimizer_Adam.apply_gradients(zip(grads, trainable_vars))        
        return loss
        

class ASDM(TINN):
    def __init__(self,
                 *arg,
                 **kwargs):
        super().__init__(*arg, **kwargs)
        
        
        self.sigma_a = tf.Variable([0.0], dtype=tf.float64,
                                   name="sigma_a",
                                   constraint= lambda z: tf.clip_by_value(z, 0, 1e10))
        self.sigma_s = tf.Variable([1.00], dtype=tf.float64, 
                                   name="sigma_s",
                                   constraint= lambda z: tf.clip_by_value(z, 0, 1e10))
        self.mu_a = tf.Variable([1.00], dtype=tf.float64, 
                                name="mu_a",
                                constraint= lambda z: tf.clip_by_value(z, 0, 1e10))
        self.rho_a = tf.Variable([1.00], dtype=tf.float64, 
                                 name="rho_a",
                                 constraint= lambda z: tf.clip_by_value(z, 0, 1e10))
        self.rho_s = tf.Variable([1.00], dtype=tf.float64, 
                                 name="rho_s",
                                 constraint= lambda z: tf.clip_by_value(z, 0, 1e10))
        self.kappa_a = tf.Variable([1.00], dtype=tf.float64,
                                   name="kappa_a",
                                   constraint= lambda z: tf.clip_by_value(z, 0, 1e10))
        
    def model_vars(self):
        return [self.sigma_a,
                self.sigma_s,
                self.mu_a,
                self.rho_a,
                self.rho_s,
                self.kappa_a
               ]
    
    def pde_residuals(self, pde_inputs, pde_outputs, partials_1, partials_2):    
        
        a = pde_outputs[:, 0]
        s = pde_outputs[:, 1]
        
        a_x = partials_1[0][:, 0]
        a_y = partials_1[0][:, 1]
        a_t = partials_1[0][:, 2]
        
        a_xx = partials_2[0][:, 0]
        a_yy = partials_2[0][:, 1]
        
        
        s_x = partials_1[1][:, 0]
        s_y = partials_1[1][:, 1]
        s_t = partials_1[1][:, 2]
        
        s_xx = partials_2[1][:, 0]
        s_yy = partials_2[1][:, 1]
        
        sigma_a = self.sigma_a
        sigma_s = self.sigma_s
        mu_a = self.mu_a
        rho_a = self.rho_a
        rho_s = self.rho_s
        kappa_a = self.kappa_a
        
        one_1 = tf.constant(1.0, dtype=tf.float64)
        f = a*a*s/(one_1 + kappa_a*a*a)
        f_a = a_t - (a_xx + a_yy) - rho_a*f + mu_a*a - sigma_a
        f_s = s_t - (s_xx + s_yy) + rho_s*f - sigma_s
        
        return tf.concat([tf.expand_dims(f_a,axis=1), 
                          tf.expand_dims(f_s,axis=1),], axis = 1)
        

In [54]:
def lower_upper_bounds(inputs_of_inputs):
    """Find the lower and upper bounds of inputs
    
       inputs_of_inputs: a list of tensors that their axis one have the same number 
                         of columns
    """
           
    inputs_dim = np.asarray(inputs_of_inputs[0]).shape[1]
    lb = np.array([np.inf] * inputs_dim)
    ub = np.array([-np.inf] * inputs_dim)
    for i, inputs in enumerate(inputs_of_inputs):        
        assert inputs_dim == np.asarray(inputs).shape[1]
        lb = np.amin(np.c_[inputs.min(0), lb], 1)
        ub = np.amax(np.c_[inputs.max(0), ub], 1)
        
    return lb, ub
    
def normalise_inputs(inputs_of_inputs):
    """Scales the values along axis 1 to [-1, 1]
    
       inputs_of_inputs: a list of tensors that their axis one have the same number 
                         of columns
    """
    if type(inputs_of_inputs) is not list:
        inputs_of_inputs = [inputs_of_inputs]        
            
    lb, ub = lower_upper_bounds(inputs_of_inputs)
    return [2.0*(inputs-lb)/(ub-lb) - 1.0 for inputs in inputs_of_inputs]    

In [48]:
layers = [3, 10, 10, 10, 2]
sample_xyt  = np.array([ [0, 0, 0],
                         [0, 1, 0],
                         [1, 0, 0],
                         [1, 1, 0],
                         [0, 0, 1],
                         [0, 1, 1],
                         [1, 0, 1],
                         [1, 1, 1],
                         [0, 0, 2],
                         [0, 1, 2],
                         [1, 0, 2],
                         [1, 1, 2]
                       ])

#lb, ub = lower_upper_bounds(sample_xyt)
#print(normalise_inputs([sample_xyt, sample_xyt.copy()*2]))

model = NN(layers)

In [29]:
#from tensorflow.python.framework.ops import disable_eager_execution

#disable_eager_execution()
i = tf.constant(sample_xyt*1.0)
o, p1, p2 = model(i, True)

RuntimeError: Exception encountered when calling layer "nn_7" (type NN).

tf.gradients is not supported when eager execution is enabled. Use tf.GradientTape instead.

Call arguments received by layer "nn_7" (type NN):
  • inputs=tf.Tensor(shape=(12, 3), dtype=float32)
  • grads=True

In [426]:
p1

[<tf.Tensor 'nn_83/gradients/nn_83/StatefulPartitionedCall_grad/PartitionedCall:0' shape=(12, 3) dtype=float64>,
 <tf.Tensor 'nn_83/gradients_1/nn_83/StatefulPartitionedCall_grad/PartitionedCall:0' shape=(12, 3) dtype=float64>]

In [49]:
layers = [3, 64, 64, 64, 64, 2]
#layers = [3, 128, 128, 128, 128, 2]

#layers = [3, 100, 100, 100, 100, 2]

# Load Data
import os
data_path = os.path.abspath("turing.npy")
with open(data_path, 'rb') as f:
    data = np.load(f)
    
data_path = os.path.abspath("turing_t.npy")
with open(data_path, 'rb') as f:
    t_star = np.load(f) 
    
T = t_star.shape[0]    
    
L = 50
x_size = data.shape[1]
y_size = data.shape[2]
N = x_size*y_size
x_domain = L*np.linspace(0,1,x_size)
y_domain = L*np.linspace(0,1,y_size)

X,Y = np.meshgrid(x_domain, y_domain, sparse=False, indexing='ij')
XX = np.tile(X.flatten(), T) # N x T
YY = np.tile(Y.flatten(), T) # N x T
TT = np.repeat(t_star[-T:], N) # T x N

AA = np.einsum('ijk->kij', data[0, :, :, -T:]).flatten() # N x T
SS = np.einsum('ijk->kij', data[1, :, :, -T:]).flatten() # N x T


x = XX[:, np.newaxis] # NT x 1
y = YY[:, np.newaxis] # NT x 1
t = TT[:, np.newaxis] # NT x 1

a = AA[:, np.newaxis] # NT x 1
s = SS[:, np.newaxis] # NT x 1

boundary_x_LB = np.concatenate((x_domain, 
                                np.repeat(x_domain[0], y_size)))
boundary_x_TR = np.concatenate((x_domain, 
                                np.repeat(x_domain[-1], y_size))) 

boundary_y_LB = np.concatenate((np.repeat(y_domain[0], x_size),
                                y_domain))
boundary_y_TR = np.concatenate((np.repeat(y_domain[-1], x_size),
                                y_domain)) 

boundary_XX_LB = np.tile(boundary_x_LB.flatten(), T)[:, np.newaxis] # (x_size + y_size) x T, 1
boundary_XX_TR = np.tile(boundary_x_TR.flatten(), T)[:, np.newaxis] # (x_size + y_size) x T, 1
boundary_YY_LB = np.tile(boundary_y_LB.flatten(), T)[:, np.newaxis] # (x_size + y_size) x T, 1
boundary_YY_TR = np.tile(boundary_y_TR.flatten(), T)[:, np.newaxis] # (x_size + y_size) x T, 1
boundary_TT = np.repeat(t_star[-T:], (x_size + y_size))[:, np.newaxis] # T x (x_size + y_size), 1


def create_dataset(training_data_size =  T*16,
                   pde_data_size =  (T*N)//(32),
                   boundary_data_size = ((x_size + y_size)*T)//(8),
                   with_boundary = True,
                   signal_to_noise = 0):
    
    ##########################################
    # Including noise
    if signal_to_noise > 0:
        signal_amp_a = (np.max(AA)-np.min(AA))/2.0
        signal_amp_s = (np.max(SS)-np.min(SS))/2.0  
        sigma_a =  signal_amp_a*signal_to_noise
        sigma_s =  signal_amp_s*signal_to_noise
    # Observed data
    idx_data = np.random.choice(N*T, training_data_size, replace=False)
    # PDE colocations
    idx_pde = np.random.choice(N*T, pde_data_size, replace=False)
    # Periodic boundary condition
    idx_boundary = np.random.choice((x_size + y_size)*T, boundary_data_size, replace=False)
    
    ret = {'x_obs': x[idx_data,:],
            'y_obs': y[idx_data,:],
            't_obs': t[idx_data,:],
            'a_obs': a[idx_data,:],
            's_obs': s[idx_data,:],
            'x_pde':   x[idx_pde,:],
            'y_pde':   y[idx_pde,:],
            't_pde':   t[idx_pde,:]}
    
    if signal_to_noise > 0:        
        ret['a_obs'] += sigma_a * np.random.randn(len(idx_data), a.shape[1])
        ret['s_obs'] += sigma_s * np.random.randn(len(idx_data), s.shape[1])
    
    if with_boundary:
        ret = {**ret,
               **{'x_boundary_LB': boundary_XX_LB[idx_boundary],
                  'x_boundary_TR': boundary_XX_TR[idx_boundary],
                  'y_boundary_LB': boundary_YY_LB[idx_boundary],
                  'y_boundary_TR': boundary_YY_TR[idx_boundary],
                  't_boundary_LB': boundary_TT[idx_boundary],
                  't_boundary_TR': boundary_TT[idx_boundary]}
              }
    return ret



In [50]:
model_params_1 = {'training_data_size': T*16,
                'pde_data_size': (T*N)//(32),
                'boundary_data_size':((x_size + y_size)*T)//(8)}

dataset = create_dataset(**model_params_1)

In [69]:
asdm = ASDM(layers, 
           inputs_obs = np.c_[dataset['x_obs'], dataset['y_obs'], dataset['t_obs']],
           output_obs = np.c_[dataset['a_obs'], dataset['s_obs']]
            )

In [70]:
asdm.train(400, T*4, print_iter=100)

Epoch: 0, loss:35680.41, 
Time:1.70

Epoch: 100, loss:2178.56, 
Time:6.64

Epoch: 200, loss:2176.05, 
Time:6.57

Epoch: 300, loss:2176.39, 
Time:6.57



In [71]:
asdm.model_vars()

[<tf.Variable 'sigma_a:0' shape=(1,) dtype=float64, numpy=array([0.10185468])>,
 <tf.Variable 'sigma_s:0' shape=(1,) dtype=float64, numpy=array([0.83944022])>,
 <tf.Variable 'mu_a:0' shape=(1,) dtype=float64, numpy=array([0.88263012])>,
 <tf.Variable 'rho_a:0' shape=(1,) dtype=float64, numpy=array([1.12305397])>,
 <tf.Variable 'rho_s:0' shape=(1,) dtype=float64, numpy=array([1.19049248])>,
 <tf.Variable 'kappa_a:0' shape=(1,) dtype=float64, numpy=array([0.82351467])>]